In [11]:
from importlib.metadata import version

import torch
from torch.distributed.autograd import context

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.5.1
tiktoken version: 0.8.0


In [12]:
from torch.utils.data import Dataset

class GPTDataset(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.inputs_ids = []
        self.targets_ids = []
        
        #Tokenize the entire text
        token_ids = tokenizer.encode(txt,allowed_special={"<|endoftext|>"})
        
        #Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            
            self.inputs_ids.append(torch.tensor(input_chunk))
            self.targets_ids.append(torch.tensor(target_chunk))
    
    def __getitem__(self, idx):
        return self.inputs_ids[idx], self.targets_ids[idx]
    
    def __len__(self):
        return len(self.inputs_ids)

In [13]:
import tiktoken
from torch.utils.data import DataLoader

def create_dataloader(txt, batch_size=4, max_length=256, stride=128, shuffle=True,drop_last=True, num_workers=0):
    
    #Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    #create dataset
    dataset = GPTDataset(txt, tokenizer, max_length, stride)
    
    #create dataloader
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)
    
    return dataloader

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/19.webp" width="400px">

In [14]:
import torch
# Use the components that built before to implement the input embedding pipeline

#read text
with open("the-verdict.txt","r", encoding="utf-8") as f:
    raw_text = f.read()

vocab_size = 50257
output_dim = 256
# context_length: This represents the maximum sequence length that the model can process. It ensures that for any position from 0 to context_length - 1, a unique positional embedding is available.
context_length = 1024

#token_embedding_layer
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

#pos_embedding_layer
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)


#token embeddings, pos embeddings, input embeddings
max_length = 4
dataloader = create_dataloader(raw_text, batch_size=8, max_length=max_length, stride=max_length)

for batch in dataloader:
    x, y = batch
    
    # Shape: (batch_size, max_length, output_dim)
    token_embeddings = token_embedding_layer(x)
    
    # Shape: (max_length, output_dim) (broadcasted to match token_embeddings).
    pos_embeddings = pos_embedding_layer(torch.arange(max_length))
    
    input_embeddings = token_embeddings + pos_embeddings
    
    break
    
print(input_embeddings.shape)

torch.Size([8, 4, 256])
